# MCP (Model Context Protocol) - Esempio Minimale con Ollama

Questo notebook dimostra l'utilizzo di MCP con un server locale Ollama usando il modello llama3.2:1b.

## Prerequisiti
- Ollama installato e in esecuzione
- Modello llama3.2:1b scaricato
- Librerie Python: `mcp`, `ollama`, `asyncio`

**IMPORTANTE**: Su Windows, assicurati di aver attivato il venv Python:
```bash
source .venv/Scripts/activate.ps1
```

In [1]:
# Installazione delle dipendenze necessarie
# Eseguire solo se le librerie non sono già installate
import subprocess
import sys

def install_package(package):
    try:
        __import__(package)
    except ImportError:
        print(f"Installazione di {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} installato!")

# Lista delle dipendenze
dependencies = ["ollama", "mcp", "pydantic"]

for dep in dependencies:
    install_package(dep)

print("✅ Tutte le dipendenze sono installate!")

✅ Tutte le dipendenze sono installate!


In [2]:
import asyncio
import json
import subprocess
from typing import Any, Dict, List
import ollama
from mcp import types
from mcp.server import Server
from mcp.client.session import ClientSession
from mcp.client.stdio import stdio_client

## 1. Configurazione Ollama

Verifichiamo che Ollama sia in esecuzione e che il modello llama3.2:1b sia disponibile.

In [3]:
# Verifica disponibilità Ollama e modello
try:
    # Controllo se Ollama è in esecuzione
    models_response = ollama.list()
    print("Ollama è in esecuzione!")
    print(f"Risposta completa: {models_response}")
    
    # Estrazione sicura dei modelli
    if 'models' in models_response:
        models_list = models_response['models']
        model_names = []
        
        for model in models_list:
            # Gestione di diverse strutture dati possibili
            if isinstance(model, ollama.ListResponse.Model):
                if 'name' in model:
                    model_names.append(model['name'])
                elif 'model' in model:
                    model_names.append(model['model'])
            elif isinstance(model, str):
                model_names.append(model)
        
        print(f"Modelli disponibili: {model_names}")
        
        # Verifica se llama3.2:1b è disponibile
        llama_found = any('llama3.2:1b' in name for name in model_names)
        if llama_found:
            print("✅ Modello llama3.2:1b trovato!")
        else:
            print("⚠️ Modello llama3.2:1b non trovato. Scaricamento in corso...")
            try:
                ollama.pull('llama3.2:1b')
                print("✅ Modello llama3.2:1b scaricato!")
            except Exception as pull_error:
                print(f"❌ Errore durante il download: {pull_error}")
                print("Prova a scaricare manualmente con: ollama pull llama3.2:1b")
    else:
        print("❌ Struttura risposta inaspettata da Ollama")
        print("Prova a eseguire 'ollama list' dal terminale per verificare")
        
except Exception as e:
    print(f"❌ Errore con Ollama: {e}")
    print(f"Tipo errore: {type(e)}")
    print("Possibili soluzioni:")
    print("1. Verifica che Ollama sia installato: ollama --version")
    print("2. Avvia il servizio Ollama se non è in esecuzione")
    print("3. Su Windows: verifica che il servizio Ollama sia attivo")

Ollama è in esecuzione!
Risposta completa: models=[Model(model='gemma3:270m', modified_at=datetime.datetime(2025, 9, 3, 10, 31, 25, 795477, tzinfo=TzInfo(UTC)), digest='e7d36fb2c3b3293cfe56d55889867a064b3a2b22e98335f2e6e8a387e081d6be', size=291554930, details=ModelDetails(parent_model='', format='gguf', family='gemma3', families=['gemma3'], parameter_size='268.10M', quantization_level='Q8_0'))]
Modelli disponibili: ['gemma3:270m']
⚠️ Modello llama3.2:1b non trovato. Scaricamento in corso...
✅ Modello llama3.2:1b scaricato!


## 2. Creazione MCP Server

Creiamo un server MCP semplice con alcuni tool di esempio.

In [4]:
class SimpleMCPServer:
    def __init__(self):
        self.server = Server("simple-mcp-server")
        self.tools_list = []
        self.setup_tools()
    
    def setup_tools(self):
        # Definiamo i tool disponibili
        self.tools_list = [
            types.Tool(
                name="calculator",
                description="Esegue calcoli matematici semplici",
                inputSchema={
                    "type": "object",
                    "properties": {
                        "expression": {
                            "type": "string",
                            "description": "Espressione matematica da calcolare"
                        }
                    },
                    "required": ["expression"]
                }
            ),
            types.Tool(
                name="text_analyzer",
                description="Analizza un testo e restituisce statistiche",
                inputSchema={
                    "type": "object",
                    "properties": {
                        "text": {
                            "type": "string",
                            "description": "Testo da analizzare"
                        }
                    },
                    "required": ["text"]
                }
            )
        ]
        
        # Registriamo i handler
        @self.server.list_tools()
        async def handle_list_tools() -> List[types.Tool]:
            """Lista dei tool disponibili"""
            return self.tools_list
        
        @self.server.call_tool()
        async def handle_call_tool(name: str, arguments: Dict[str, Any]) -> List[types.TextContent]:
            """Gestisce le chiamate ai tool"""
            return await self.execute_tool(name, arguments)
    
    async def execute_tool(self, name: str, arguments: Dict[str, Any]) -> List[types.TextContent]:
        """Esegue il tool specificato"""
        if name == "calculator":
            try:
                expression = arguments["expression"]
                # Valutazione sicura di espressioni matematiche semplici
                result = eval(expression, {"__builtins__": {}}, {
                    "abs": abs, "round": round, "min": min, "max": max,
                    "sum": sum, "pow": pow, "len": len, "+": lambda x, y: x + y,
                    "-": lambda x, y: x - y, "*": lambda x, y: x * y, "/": lambda x, y: x / y
                })
                return [types.TextContent(
                    type="text",
                    text=f"Risultato: {expression} = {result}"
                )]
            except Exception as e:
                return [types.TextContent(
                    type="text",
                    text=f"Errore nel calcolo: {str(e)}"
                )]
        
        elif name == "text_analyzer":
            text = arguments["text"]
            words = text.split()
            chars = len(text)
            sentences = text.count('.') + text.count('!') + text.count('?')
            
            analysis = {
                "caratteri": chars,
                "parole": len(words),
                "frasi": sentences,
                "caratteri_medi_per_parola": round(chars / len(words), 2) if words else 0
            }
            
            return [types.TextContent(
                type="text",
                text=f"Analisi del testo:\n{json.dumps(analysis, indent=2, ensure_ascii=False)}"
            )]
        
        else:
            return [types.TextContent(
                type="text",
                text=f"Tool '{name}' non riconosciuto"
            )]
    
    async def get_tools(self):
        """Restituisce la lista dei tool disponibili"""
        return self.tools_list
    
    async def call_tool_method(self, name: str, arguments: Dict[str, Any]):
        """Wrapper per chiamare i tool"""
        return await self.execute_tool(name, arguments)

# Creazione del server
mcp_server = SimpleMCPServer()
print("✅ MCP Server creato con successo!")

✅ MCP Server creato con successo!


## 3. Funzione Helper per Ollama

Creiamo una funzione per interagire con Ollama in modo semplice.

In [5]:
def chat_with_ollama(prompt: str, model: str = "llama3.2:1b", tools_available: bool = False) -> str:
    """
    Interagisce con Ollama usando il modello specificato
    """
    try:
        if tools_available:
            # Aggiungiamo informazioni sui tool disponibili nel prompt
            system_prompt = """
            Sei un assistente AI con accesso ai seguenti tool:
            1. calculator: per calcoli matematici (usa l'argomento 'expression')
            2. text_analyzer: per analizzare testi (usa l'argomento 'text')
            
            Quando l'utente chiede qualcosa che può essere risolto con questi tool,
            rispondi con il formato: TOOL_CALL: nome_tool {"argomento": "valore"}
            """
            full_prompt = f"{system_prompt}\n\nUtente: {prompt}"
        else:
            full_prompt = prompt
            
        response = ollama.chat(
            model=model,
            messages=[{'role': 'user', 'content': full_prompt}]
        )
        return response['message']['content']
    except Exception as e:
        error_msg = f"Errore nella comunicazione con Ollama: {str(e)}"
        print(f"Debug - {error_msg}")
        
        # Suggerimenti per errori comuni
        if "model" in str(e).lower():
            error_msg += "\nSuggerimento: Verifica che il modello sia disponibile con 'ollama list'"
        elif "connection" in str(e).lower():
            error_msg += "\nSuggerimento: Verifica che Ollama sia in esecuzione"
            
        return error_msg

# Test della funzione con gestione errori migliorata
print("Test di comunicazione con Ollama...")
test_response = chat_with_ollama("Ciao! Come stai?")
print(f"Risposta di Ollama: {test_response}")

Test di comunicazione con Ollama...
Risposta di Ollama: Ciao! Io sto bene, grazie. Sono felice di poterti aiutare con qualsiasi cosa. Come posso aiutarti oggi?


## 4. Simulazione Client MCP

Creiamo un client simulato che può utilizzare i tool del server.

In [6]:
class MCPClient:
    def __init__(self, server):
        self.server = server
        self.available_tools = []
    
    async def initialize(self):
        """Inizializza il client e ottiene la lista dei tool"""
        try:
            self.available_tools = await self.server.get_tools()
            print(f"Tool disponibili: {[tool.name for tool in self.available_tools]}")
        except Exception as e:
            print(f"Errore nell'inizializzazione del client: {e}")
            # Fallback: usa la lista dei tool direttamente
            self.available_tools = self.server.tools_list
            print(f"Tool disponibili (fallback): {[tool.name for tool in self.available_tools]}")
    
    async def call_tool(self, tool_name: str, arguments: Dict[str, Any]):
        """Chiama un tool specifico"""
        try:
            result = await self.server.call_tool_method(tool_name, arguments)
            return result[0].text if result else "Nessun risultato"
        except Exception as e:
            return f"Errore nella chiamata al tool: {str(e)}"
    
    def parse_tool_call(self, llm_response: str):
        """Analizza la risposta del LLM per identificare chiamate ai tool"""
        if "TOOL_CALL:" in llm_response:
            try:
                # Estrae il nome del tool e gli argomenti
                parts = llm_response.split("TOOL_CALL:")[1].strip()
                tool_name = parts.split(" ")[0]
                args_str = parts.split(" ", 1)[1] if " " in parts else "{}"
                arguments = json.loads(args_str)
                return tool_name, arguments
            except Exception as e:
                print(f"Errore nel parsing della chiamata tool: {e}")
                return None, None
        return None, None

# Inizializzazione del client
mcp_client = MCPClient(mcp_server)
await mcp_client.initialize()

Tool disponibili: ['calculator', 'text_analyzer']


## 5. Workflow Completo

Ora combiniamo tutto per creare un workflow completo che usa Ollama con i tool MCP.

In [7]:
async def complete_workflow(user_input: str):
    """
    Workflow completo: LLM + MCP Tools
    """
    print(f"🤖 Utente: {user_input}")
    
    # 1. Ottieni risposta da Ollama con informazioni sui tool
    llm_response = chat_with_ollama(user_input, tools_available=True)
    print(f"💭 LLM risposta: {llm_response}")
    
    # 2. Controlla se il LLM vuole usare un tool
    tool_name, arguments = mcp_client.parse_tool_call(llm_response)
    
    if tool_name:
        print(f"🔧 Chiamata tool: {tool_name} con argomenti: {arguments}")
        
        # 3. Esegui il tool
        tool_result = await mcp_client.call_tool(tool_name, arguments)
        print(f"📊 Risultato tool: {tool_result}")
        
        # 4. Ottieni una risposta finale dal LLM con il risultato del tool
        final_prompt = f"L'utente ha chiesto: {user_input}\nHo usato il tool {tool_name} e il risultato è: {tool_result}\nFornisci una risposta completa e utile."
        final_response = chat_with_ollama(final_prompt)
        print(f"✅ Risposta finale: {final_response}")
    else:
        print(f"💬 Risposta diretta (senza tool): {llm_response}")

# Test del workflow completo
print("=== Test 1: Calcolo matematico ===")
await complete_workflow("Quanto fa 15 * 23 + 7?")

print("\n=== Test 2: Analisi testo ===")
await complete_workflow("Puoi analizzare questo testo: 'Il machine learning è una branca dell'intelligenza artificiale. Permette ai computer di imparare dai dati.'?")

print("\n=== Test 3: Domanda generale ===")
await complete_workflow("Spiegami cosa sono i protocolli di comunicazione.")

=== Test 1: Calcolo matematico ===
🤖 Utente: Quanto fa 15 * 23 + 7?
💭 LLM risposta: TOOL_CALL: calculator {"value": "41875"}
🔧 Chiamata tool: calculator con argomenti: {'value': '41875'}
📊 Risultato tool: Errore nel calcolo: 'expression'
✅ Risposta finale: Per risolvere questo problema, possiamo following i passaggi:

1.  Calcolare il prodotto del numero 15 per cui siamo chiedendo il risultato di 15 * 23.
2.  Aggiungere la somma di 7 al risultato del calcolo precedente.

Calcolando i due risultati:

15 * 23 = 345

Aggiungiamo poi 7 ai risultati precedentemente calcolati:

345 + 7 = 352

=== Test 2: Analisi testo ===
🤖 Utente: Puoi analizzare questo testo: 'Il machine learning è una branca dell'intelligenza artificiale. Permette ai computer di imparare dai dati.'?
💭 LLM risposta: TOOL_CALL: text_analyzer {"argomento": "testo per analisi di linguaggio naturale"}
🔧 Chiamata tool: text_analyzer con argomenti: {'argomento': 'testo per analisi di linguaggio naturale'}
📊 Risultato tool: Error

## 6. Test Interattivo

Creiamo un semplice loop interattivo per testare il sistema.

In [ ]:
async def interactive_chat():
    """
    Chat interattiva con MCP + Ollama
    """
    print("🚀 Chat MCP + Ollama avviata!")
    print("Tool disponibili: calculator, text_analyzer")
    print("Digita 'quit' per uscire.\n")
    
    while True:
        try:
            user_input = input("Tu: ")
            if user_input.lower() in ['quit', 'exit', 'q']:
                print("👋 Chat terminata!")
                break
            
            if user_input.strip():
                await complete_workflow(user_input)
                print("-" * 50)
        except KeyboardInterrupt:
            print("\n👋 Chat terminata!")
            break
        except Exception as e:
            print(f"❌ Errore: {e}")

# Per avviare la chat interattiva, decommentare la riga seguente:
await interactive_chat()

print("✅ Notebook completato! Decommentare l'ultima riga per avviare la chat interattiva.")

🚀 Chat MCP + Ollama avviata!
Tool disponibili: calculator, text_analyzer
Digita 'quit' per uscire.



## Risoluzione Problemi

Se riscontri errori, prova i seguenti comandi dal terminale:

### Windows (PowerShell)
```powershell
# Attiva il virtual environment
source .venv/Scripts/activate.ps1

# Verifica Ollama
ollama --version
ollama list
ollama serve  # Se il servizio non è attivo

# Scarica il modello se necessario
ollama pull llama3.2:1b
```

### Verifica installazione
```bash
python -c "import ollama; print('Ollama library OK')"
python -c "import mcp; print('MCP library OK')"
```

## Conclusioni

Questo notebook dimostra:

1. **Setup MCP Server**: Creazione di un server con tool personalizzati
2. **Integrazione Ollama**: Uso del modello llama3.2:1b locale
3. **Workflow completo**: Combinazione di LLM e tool MCP
4. **Tool disponibili**:
   - `calculator`: per calcoli matematici
   - `text_analyzer`: per analisi di testi

### Prossimi passi:
- Aggiungere più tool personalizzati
- Implementare persistenza dei dati
- Migliorare il parsing delle chiamate ai tool
- Aggiungere validazione degli input